<a href="https://colab.research.google.com/github/Prasunsk/Analog-LTSpice-Neuron/blob/master/Training_is_GSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
import json
import numpy as np
from numpy import moveaxis
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
data_path = '/gdrive/My Drive/Speech_Processing/Json_files/features.json'
SAVED_MODEL_PATH = '/gdrive/My Drive/Speech_Processing/fullset_gsc_model.h5'
EPOCHS = 40
BATCH_SIZE = 32
PATIENCE = 5
LEARNING_RATE = 0.0001

In [4]:
# def load_data(data_path):
#     """Loads training dataset from json file.

#     :param data_path (str): Path to json file containing data
#     :return X (ndarray): Inputs
#     :return y (ndarray): Targets

#     """
with open(data_path, "r") as fp:
    data = json.load(fp)
X=np.zeros((11016,3,13,32))
for i in range(len(X)):
    X[i][0] = np.array([data["MFCCs"][i]])
    X[i][1] = np.array([data["MFCC_delta"][i]])
    X[i][2] = np.array([data["MFCC_delta_delta"][i]])
    y = np.array(data["labels"])
        
X = moveaxis(X, 1, 3)
print(X.shape) 
print("Training sets loaded!")
    # return X, y

(11016, 13, 32, 3)
Training sets loaded!


In [5]:
len(data['MFCCs'])

11016

In [6]:
# def prepare_dataset(data_path, test_size=0.2, validation_size=0.2):
#     """Creates train, validation and test sets.

#     :param data_path (str): Path to json file containing data
#     :param test_size (flaot): Percentage of dataset used for testing
#     :param validation_size (float): Percentage of train set used for cross-validation

#     :return X_train (ndarray): Inputs for the train set
#     :return y_train (ndarray): Targets for the train set
#     :return X_validation (ndarray): Inputs for the validation set
#     :return y_validation (ndarray): Targets for the validation set
#     :return X_test (ndarray): Inputs for the test set
#     :return X_test (ndarray): Targets for the test set
#     """

    # load dataset
# X, y = load_data(data_path)
# print(len(X))

test_size=0.2
validation_size=0.2
    # create train, validation, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # return X_train, y_train, X_validation, y_validation, X_test, y_test

In [9]:
len(data['MFCCs'])

11016

In [17]:
def build_model(input_shape, loss="sparse_categorical_crossentropy", learning_rate=0.0001):
#     """Build neural network using keras.

#     :param input_shape (tuple): Shape of array representing a sample train. E.g.: (44, 13, 1)
#     :param loss (str): Loss function to use
#     :param learning_rate (float):

#     :return model: TensorFlow model
#     """

#     # build network architecture using convolutional layers
    model = tf.keras.models.Sequential()

    # 1st conv layer
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape,data_format="channels_last",kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

    # 2nd conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',data_format="channels_last", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

    # # 3rd conv layer
    # model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu',
    #                                  kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    # model.add(tf.keras.layers.BatchNormalization())
    # model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

    # flatten output and feed into dense layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(120, activation='relu'))
    tf.keras.layers.Dropout(0.3)

    # softmax output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    optimiser = tf.optimizers.Adam(learning_rate=learning_rate)

    # compile model
    model.compile(optimizer=optimiser,
                  loss=loss,
                  metrics=["accuracy"])
        # print model parameters on console
    model.summary()

    return model


In [11]:
def train(model, epochs, batch_size, patience, X_train, y_train, X_validation, y_validation):
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", min_delta=0.001, patience=patience)
    # train model
    history = model.fit(X_train, y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_validation, y_validation),callbacks=[earlystop_callback])
    return history

In [18]:
    # generate train, validation and test sets
# X_train, y_train, X_validation, y_validation, X_test, y_test = prepare_dataset(data_path)

    # create network
input_shape = (X_train.shape[1], X_train.shape[2], 1)   #(#sequence, #coefficients=13, 1)
model = build_model(input_shape, learning_rate=LEARNING_RATE)

    # train network
history = train(model, EPOCHS, BATCH_SIZE, PATIENCE, X_train, y_train, X_validation, y_validation)

    # plot accuracy/loss for training/validation set as a function of the epochs


    # evaluate network on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("\nTest loss: {}, test accuracy: {}".format(test_loss, 100*test_acc))

    # save model
model.save(SAVED_MODEL_PATH)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 11, 30, 64)        640       
_________________________________________________________________
batch_normalization_6 (Batch (None, 11, 30, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 15, 64)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 13, 32)         18464     
_________________________________________________________________
batch_normalization_7 (Batch (None, 4, 13, 32)         128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 2, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 448)              

ValueError: ignored

In [15]:
X.shape

(11016, 13, 32, 3)

In [ ]:
# fig1 = plt.figure(1)
from google.colab import files
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")
plt.title("Accuracy evaluation")
plt.savefig(f'/content/acc.png')

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.legend(loc="upper right")
plt.title("Loss evaluation")
plt.savefig(f'/content/loss.png')